# Setup DatasetUploader object and read in file as AnnData

In [1]:
import os, sys
import pandas as pd
import scanpy.api as sc

#Add uploader to PATH and import it
sys.path.append('/home/dolley/gear/lib')
from gear.datasetuploader import FileType, DatasetUploader


test_file = '/home/dolley/gear/tests/base_template.xlsx' # basic test dataset

#Initialize uploader object
dataset = DatasetUploader.upload_dataset('xlsx')

#Read file into uploader object
dataset._read_file(test_file)
dataset.adata

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


AnnData object with n_obs × n_vars = 18 × 100 
    obs: 'cell_type', 'condition', 'replicate', 'time_point', 'time_unit'
    var: 'gene_symbol'

# Setup MetadataUploader and read in file

In [25]:
from gear.metadatauploader import MetadataUploader as uploader
from gear.uploadlogger import UploadLogger

metadata_file = '/home/dolley/gear/tests/dataset_metadata_test.xlsx'

metadata_uploader = uploader()              # Iniitialze
metadata_uploader._read_file(metadata_file) # Read in metadata file
metadata_uploader._validate_values()        # validate user inputs

#Start error logger
from gear.uploadlogger import UploadLogger
error_log = UploadLogger()

# Get list of invalid fields
error_log.log_error(metadata_uploader._list_invalid_fields())

#Print errors
print(error_log.list_errors())

#Write metadata to JSON
metadata_uploader._write_to_json(filepath='/home/dolley/demo_metadata_output.json')

Looks good. No invalid metadata fields found.


In [28]:
import json
with open('/home/dolley/demo_metadata_output.json', 'r') as j:
    test = json.load(j)
print(json.dumps(test, indent=4))

{
    "values": {
        "annotation_source": "ensembl",
        "species_assembly_version": "GRCm38",
        "description": "Some stuff was done to produce these things.",
        "geo_id": "GSE1234",
        "email": "my.name@gmail.com",
        "last_name": "Name",
        "species_annotation_release_number": 91,
        "annotation_protocol": "bowtie",
        "pubmed_id": 1234,
        "data_type": "rpkm",
        "first_name": "My",
        "experiment_type": "Rna-seq",
        "title": "A title worthy of titling a dataset",
        "species_information": "leave blank",
        "species_name": "mus musculus",
        "data_evidence": "illumina",
        "pi_contact_information": "leave blank",
        "organization": "University of the Small Island"
    }
}


# Calculate Statistics for dataset (Alex Wolf's)

In [2]:
# USES MY LOGIC - produces adata.uns['Xmean'], Xstd, Xsem, Xpval, and Xfdr
#  Each is a numpy matrix and contains duplicated values to maintain exact shape of adata.X

# dataset._add_calculated_values(dataset.adata)
# dataset.adata.uns['Xmean']

'''
dataset._add_calculated_values(dataset.adata)
dataset.adata.Xmean
array([[71.6667, 53.3333, 49.6667, ..., 69.6667, 45.3333, 48.6667],
       [71.6667, 53.3333, 49.6667, ..., 69.6667, 45.3333, 48.6667],
       [71.6667, 53.3333, 49.6667, ..., 69.6667, 45.3333, 48.6667],
       ...,
       [48.6667, 34.3333, 50.6667, ..., 55.6667, 62.3333, 57.3333],
       [48.6667, 34.3333, 50.6667, ..., 55.6667, 62.3333, 57.3333],
       [48.6667, 34.3333, 50.6667, ..., 55.6667, 62.3333, 57.3333]],
      dtype=float32)
'''

# USES Alex Wolf's logic
adata = dataset.adata
X = adata.X
obs = adata.obs

#Prep expression expression for stat calculations
df = pd.DataFrame(X)  # does not allocate new memory if X is an array, so this efficient
df['cell_type'] = obs['cell_type'].values  # if not using assign, no copy is made
df['condition'] = obs['condition'].values  # if not using assign, no copy is made
df['time_point'] = obs['time_point'].values  # if not using assign, no copy is made
df_grouped = df.groupby(['cell_type', 'condition', 'time_point'])

#Calculate Mean, Standard Deviation and Standard Error of the Mean
# add resultant dataframes to adata object
Xmean = df_grouped.mean()
Xstd = df_grouped.std()
Xsem = df_grouped.sem()

#Stores statistics as numpy arrays when file is written. So we lose the groupings
adata.uns['Xmean'] = Xmean
adata.uns['Xstd'] = Xstd
adata.uns['Xsem'] = Xsem


print(dataset.adata.uns['Xmean'])
print(type(dataset.adata.uns['Xmean']))

                                       0          1          2          3   \
cell_type condition time_point                                               
utricle   control   0           71.666664  53.333332  49.666668  67.000000   
                    24          26.333334  42.000000  66.333336  40.000000   
                    48          65.666664  61.333332  36.666668  66.333336   
          treated   0           46.666668  77.000000  54.666668  32.333332   
                    24          53.000000  42.333332  22.666666  32.000000   
                    48          48.666668  34.333332  50.666668  51.333332   

                                       4          5          6          7   \
cell_type condition time_point                                               
utricle   control   0           63.333332  10.333333  26.333334  55.000000   
                    24          34.000000  44.333332  48.666668  46.666668   
                    48          43.333332  57.000000  63.666668

In [13]:
adata = sc.read_h5ad('/home/dolley/git/jupyter_notebooks/TEST_base_alex.h5ad')
pd.DataFrame(adata.uns['Xmean'])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,71.666664,53.333332,49.666668,67.000000,63.333332,10.333333,26.333334,55.000000,49.000000,26.000000,...,62.000000,72.333336,39.000000,51.000000,68.000000,67.333336,71.666664,69.666664,45.333332,48.666668
1,26.333334,42.000000,66.333336,40.000000,34.000000,44.333332,48.666668,46.666668,50.000000,80.333336,...,73.000000,38.333332,83.333336,52.000000,51.666668,27.333334,53.000000,35.666668,57.666668,71.333336
2,65.666664,61.333332,36.666668,66.333336,43.333332,57.000000,63.666668,46.666668,26.333334,44.666668,...,39.333332,61.666668,50.666668,73.666664,39.333332,28.666666,80.333336,41.666668,43.333332,51.000000
3,46.666668,77.000000,54.666668,32.333332,55.666668,23.666666,42.666668,45.666668,16.000000,60.000000,...,28.333334,35.333332,52.333332,71.666664,41.000000,61.666668,32.000000,36.666668,45.333332,55.000000
4,53.000000,42.333332,22.666666,32.000000,44.000000,45.000000,37.000000,44.333332,43.666668,61.666668,...,88.666664,30.000000,49.666668,48.000000,24.666666,41.666668,25.000000,40.666668,60.000000,45.333332
5,48.666668,34.333332,50.666668,51.333332,34.333332,40.666668,46.333332,69.000000,54.666668,30.333334,...,36.666668,46.666668,73.666664,54.333332,60.333332,52.333332,67.000000,55.666668,62.333332,57.333332


In [14]:
adata = sc.read_h5ad('/home/dolley/git/jupyter_notebooks/TEST_base_dustin.h5ad')
pd.DataFrame(adata.uns['Xmean'])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,71.666702,53.333302,49.666698,67.000000,63.333302,10.333300,26.333300,55.000000,49.000000,26.000000,...,62.000000,72.333298,39.000000,51.000000,68.000000,67.333298,71.666702,69.666702,45.333302,48.666698
1,71.666702,53.333302,49.666698,67.000000,63.333302,10.333300,26.333300,55.000000,49.000000,26.000000,...,62.000000,72.333298,39.000000,51.000000,68.000000,67.333298,71.666702,69.666702,45.333302,48.666698
2,71.666702,53.333302,49.666698,67.000000,63.333302,10.333300,26.333300,55.000000,49.000000,26.000000,...,62.000000,72.333298,39.000000,51.000000,68.000000,67.333298,71.666702,69.666702,45.333302,48.666698
3,46.666698,77.000000,54.666698,32.333302,55.666698,23.666700,42.666698,45.666698,16.000000,60.000000,...,28.333300,35.333302,52.333302,71.666702,41.000000,61.666698,32.000000,36.666698,45.333302,55.000000
4,46.666698,77.000000,54.666698,32.333302,55.666698,23.666700,42.666698,45.666698,16.000000,60.000000,...,28.333300,35.333302,52.333302,71.666702,41.000000,61.666698,32.000000,36.666698,45.333302,55.000000
5,46.666698,77.000000,54.666698,32.333302,55.666698,23.666700,42.666698,45.666698,16.000000,60.000000,...,28.333300,35.333302,52.333302,71.666702,41.000000,61.666698,32.000000,36.666698,45.333302,55.000000
6,26.333300,42.000000,66.333298,40.000000,34.000000,44.333302,48.666698,46.666698,50.000000,80.333298,...,73.000000,38.333302,83.333298,52.000000,51.666698,27.333300,53.000000,35.666698,57.666698,71.333298
7,26.333300,42.000000,66.333298,40.000000,34.000000,44.333302,48.666698,46.666698,50.000000,80.333298,...,73.000000,38.333302,83.333298,52.000000,51.666698,27.333300,53.000000,35.666698,57.666698,71.333298
8,26.333300,42.000000,66.333298,40.000000,34.000000,44.333302,48.666698,46.666698,50.000000,80.333298,...,73.000000,38.333302,83.333298,52.000000,51.666698,27.333300,53.000000,35.666698,57.666698,71.333298
9,53.000000,42.333302,22.666700,32.000000,44.000000,45.000000,37.000000,44.333302,43.666698,61.666698,...,88.666702,30.000000,49.666698,48.000000,24.666700,41.666698,25.000000,40.666698,60.000000,45.333302


# Write dataset to file (uses uploader method)

In [7]:
# dataset._write2h5ad()

In [8]:
# adata = sc.read_h5ad('/home/dolley/git/jupyter_notebooks/9aef814d-f6e5-4e74-abf3-8a40dd35ef14.h5ad')
# adata

### --------------------------------------------------------------------------------------------------------------------------------------------------
# Simulate user searching for gene

In [16]:
#Gene symbol searched or Ensembl ID
searched_gene = "Gnai3"
searched_gene_ensembl = "ENSMUSG00000000567"

#Select a dataset
#base_template
adata = sc.read_h5ad('/home/dolley/git/jupyter_notebooks/TEST_base_dustin.h5ad')

#base_multicelltypes
# adata = sc.read_h5ad('/home/dolley/git/jupyter_notebooks/TEST_multicelltypes.h5ad')

#dataset uploaded with no 'genes' sheet.
# adata = sc.read_h5ad('/home/dolley/git/jupyter_notebooks/TEST_nogenes.h5ad')
# print(adata)

#Find the gene's index position (integer)
var = adata.var
var_index = pd.Index(var.index)

try:
    gene_index_label = var.index[var['gene_symbol'] == searched_gene].tolist()[0] #Get 1st found
    gene_index = var_index.get_loc(gene_index_label) #Get the index label's position in the index
except:
    gene_index = var_index.get_loc(searched_gene_ensembl) #Get the index label's position in the index

gene_index     

# gene_index_label = var.index[var['gene_symbol'] == searched_gene].tolist()[0] #Get 1st found
# gene_index = var_index.get_loc(gene_index_label) #Get the index label's position in the index
# gene_index 

0

# Get Xmean and Xstd from .uns and regroup

In [17]:
df_means = pd.DataFrame(adata.uns['Xmean'])
df_stds = pd.DataFrame(adata.uns['Xstd'])


df_means['cell_type'] = adata.obs['cell_type'].values
df_means['condition'] = adata.obs['condition'].values
df_means['time_point'] = adata.obs['time_point'].values

df_stds['cell_type'] = adata.obs['cell_type'].values
df_stds['condition'] = adata.obs['condition'].values
df_stds['time_point'] = adata.obs['time_point'].values

df_means = df_means.groupby(['cell_type', 'condition', 'time_point']).mean()
df_stds = df_stds.groupby(['cell_type', 'condition', 'time_point']).mean()

df_means

0          1          2          3   \
cell_type condition time_point                                               
utricle   control   0           71.666702  53.333302  49.666698  67.000000   
                    24          26.333300  42.000000  66.333298  40.000000   
                    48          65.666702  61.333302  36.666698  66.333298   
          treated   0           46.666698  77.000000  54.666698  32.333302   
                    24          53.000000  42.333302  22.666700  32.000000   
                    48          48.666698  34.333302  50.666698  51.333302   

                                       4          5          6          7   \
cell_type condition time_point                                               
utricle   control   0           63.333302  10.333300  26.333300  55.000000   
                    24          34.000000  44.333302  48.666698  46.666698   
                    48          43.333302  57.000000  63.666698  46.666698   
          treated   0           55.666698  23.666700  42.666698  45.666698   
                    24          44.000000  45.000000  37.000000  44.333302   
                    48          34.333302  40.666698  46.333302  69.000000   

                                       8          9     ...             90  \
cell_type condition time_point                          ...                  
utricle   control   0           49.000000  26.000000    ...      62.000000   
                    24          50.000000  80.333298    ...      73.000000   
                    48          26.333300  44.666698    ...      39.333302   
          treated   0           16.000000  60.000000    ...      28.333300   
                    24          43.666698  61.666698    ...      88.666702   
                    48          54.666698  30.333300    ...      36.666698   

                                       91         92         93         94  \
cell_type condition time_point                                               
utricle   control   0           72.333298  39.000000  51.000000  68.000000   
                    24          38.333302  83.333298  52.000000  51.666698   
                    48          61.666698  50.666698  73.666702  39.333302   
          treated   0           35.333302  52.333302  71.666702  41.000000   
                    24          30.000000  49.666698  48.000000  24.666700   
                    48          46.666698  73.666702  54.333302  60.333302   

                                       95         96         97         98  \
cell_type condition time_point                                               
utricle   control   0           67.333298  71.666702  69.666702  45.333302   
                    24          27.333300  53.000000  35.666698  57.666698   
                    48          28.666700  80.333298  41.666698  43.333302   
          treated   0           61.666698  32.000000  36.666698  45.333302   
                    24          41.666698  25.000000  40.666698  60.000000   
                    48          52.333302  67.000000  55.666698  62.333302   

                                       99  
cell_type condition time_point             
utricle   control   0           48.666698  
                    24          71.333298  
                    48          51.000000  
          treated   0           55.000000  
                    24          45.333302  
                    48          57.333302  

[6 rows x 100 columns]

# Find gene's expression and stats

In [18]:
# Get mean expressions for the gene
expression_means = df_means.iloc[:,gene_index]
expression_stds = df_stds.iloc[:,gene_index]
expression_means

cell_type  condition  time_point
utricle    control    0             71.666702
                      24            26.333300
                      48            65.666702
           treated    0             46.666698
                      24            53.000000
                      48            48.666698
Name: 0, dtype: float32

# Plot expression as bar graph

In [19]:
import plotly.offline as py
import plotly.graph_objs as go
import cufflinks as cf

# print("MEANS: ", expression_means)
# print(expression_means.loc['utricle', 'control', 0])
cell_types = tuple(expression_means.index.get_level_values('cell_type').unique())
conditions = tuple(expression_means.index.get_level_values('condition').unique())
time_points = tuple(expression_means.index.get_level_values('time_point').unique())
# print(cell_types)
# print(conditions)
# print(time_points)

data = []
for time_point in time_points:
    data.append(
        go.Bar(
            x=conditions,
            y=expression_means.loc[cell_types, conditions, time_point],
            error_y=dict(
                type='data',
                array=expression_stds.loc[cell_types, conditions, time_point]
            ),
            name = time_point
        )
    )
# data = []
# for cell_type in cell_types:
#     print('cell: ', cell_type)
#     for time_point in time_points:
#         print('time: ', time_point)
#         data.append(
#             go.Bar(
#                 x=conditions,
#                 y=expression_means.loc[cell_type, conditions, time_point][0],
#                 error_y=dict(
#                     type='data',
#                     array=expression_stds.loc[cell_type, conditions, time_point][0]
#                 ),
#                 name = str(time_point) + " " + cell_type
#             )
#         )

layout = go.Layout( xaxis=go.XAxis(type='category') )
fig = go.Figure( data=data, layout=layout )

# From commandline:
# url = py.plot( fig )

# From Jupyter Notebook:
py.iplot(fig)
print(fig.to_string() )

Figure(
    data=Data([
        Bar(
            x=('control', 'treated'),
            y=cell_type  condition  time_point
utricle    control    0      ..,
            error_y=ErrorY(
                array=cell_type  condition  time_point
utricle    control   ..,
                type='data'
            ),
            name=0
        ),
        Bar(
            x=('control', 'treated'),
            y=cell_type  condition  time_point
utricle    control    24     ..,
            error_y=ErrorY(
                array=cell_type  condition  time_point
utricle    control   ..,
                type='data'
            ),
            name=24
        ),
        Bar(
            x=('control', 'treated'),
            y=cell_type  condition  time_point
utricle    control    48     ..,
            error_y=ErrorY(
                array=cell_type  condition  time_point
utricle    control   ..,
                type='data'
            ),
            name=48
        )
    ]),
    layout=Layout(
        xaxi